In [1]:
import pandas as pd
import numpy as np
import time
import pymysql
# 不显示警告信息
import warnings
warnings.filterwarnings('ignore')

from pyecharts.charts import Bar3D

# 加载数据

**加载 sales.xlsx 的数据，包含5个sheet：'2015'、'2016'、'2017'、'2018'、'会员等级'**

In [2]:
sheet_names = ['2015', '2016', '2017', '2018', '会员等级']
sheet_datas = [pd.read_excel('./data/sales.xlsx', sheet_name=i, engine='openpyxl') for i in sheet_names]
sheet_datas

[              会员ID         订单号       提交日期    订单金额
 0      15278002468  3000304681 2015-01-01   499.0
 1      39236378972  3000305791 2015-01-01  2588.0
 2      38722039578  3000641787 2015-01-01   498.0
 3      11049640063  3000798913 2015-01-01  1572.0
 4      35038752292  3000821546 2015-01-01    10.1
 ...            ...         ...        ...     ...
 30769  39368100847  4281994827 2015-12-31   828.0
 30770       409757  4282010457 2015-12-31   199.0
 30771  38380526114  4282017675 2015-12-31   208.0
 30772     28074988  4282019440 2015-12-31    89.0
 30773  39460363230  4282025309 2015-12-31   719.0
 
 [30774 rows x 4 columns],
               会员ID         订单号       提交日期     订单金额
 0      39288120141  4282025766 2016-01-01    76.00
 1      39293812118  4282037929 2016-01-01  7599.00
 2      27596340905  4282038740 2016-01-01   802.00
 3      15111475509  4282043819 2016-01-01    65.00
 4      38896594001  4282051044 2016-01-01    95.00
 ...            ...         ...        ...     

**zip方法的使用**

In [3]:
# sheet_names = ['2015', '2016', '2017', '2018', '会员等级']
# sheet_datas = ['数据1', '数据2', '数据3', '数据4', '数据5']

# for a, b in zip(sheet_names, sheet_datas):
#     print(a, b)

**any方法的使用**

In [4]:
df = pd.DataFrame([
    [1, 2, 3],
    [np.nan, 4, 5],
    [4, 5, 6],
    [7, np.nan, 8]
], columns=['a', 'b', 'c'])
df

,a,b,c
0,1.0,2.0,3
1,NaN,4.0,5
2,4.0,5.0,6
3,7.0,NaN,8


In [5]:
df.isnull()

,a,b,c
0,False,False,False
1,True,False,False
2,False,False,False
3,False,True,False


In [6]:
df.isnull().any()

a     True
b     True
c    False
dtype: bool

In [7]:
df.isnull().any(axis=1)

0    False
1     True
2    False
3     True
dtype: bool

In [8]:
df.isnull().any(axis=1).sum()

2

In [9]:
for each_name, each_data in zip(sheet_names, sheet_datas):
    print('[data summary for ============={}===============]'.format(each_name))
    print('Overview:', '\n', each_data.head(4))# 展示数据前4条
    print('DESC:', '\n', each_data.describe())# 数据描述性信息
    print('NA records', each_data.isnull().any(axis=1).sum()) # 缺失值记录数    
    print('Dtypes', each_data.dtypes) # 数据类型

[data summary for =============2015===============]
Overview: 
           会员ID         订单号       提交日期    订单金额
0  15278002468  3000304681 2015-01-01   499.0
1  39236378972  3000305791 2015-01-01  2588.0
2  38722039578  3000641787 2015-01-01   498.0
3  11049640063  3000798913 2015-01-01  1572.0
DESC: 
                会员ID           订单号           订单金额
count  3.077400e+04  3.077400e+04   30774.000000
mean   2.918779e+10  4.020414e+09     960.991161
std    1.385333e+10  2.630510e+08    2068.107231
min    2.670000e+02  3.000305e+09       0.500000
25%    1.944122e+10  3.885510e+09      59.000000
50%    3.746545e+10  4.117491e+09     139.000000
75%    3.923593e+10  4.234882e+09     899.000000
max    3.954613e+10  4.282025e+09  111750.000000
NA records 0
Dtypes 会员ID             int64
订单号              int64
提交日期    datetime64[ns]
订单金额           float64
dtype: object
[data summary for =============2016===============]
Overview: 
           会员ID         订单号       提交日期    订单金额
0  39288120141  42820

# 数据预处理

**缺失值和异常数据的处理**

In [10]:
# 去除缺失值和异常值
for i, each_data in enumerate(sheet_datas[:-1]):
    sheet_datas[i] = each_data.dropna()
    sheet_datas[i] = each_data[each_data['订单金额'] > 1]
    sheet_datas[i]['max_year_date'] = each_data['提交日期'].max()

In [14]:
for each_name, each_data in zip(sheet_names, sheet_datas):
    print('[data summary for ============={}===============]'.format(each_name))
    print('Overview:', '\n', each_data.head(4))# 展示数据前4条
    print('DESC:', '\n', each_data.describe())# 数据描述性信息
    print('NA records', each_data.isnull().any(axis=1).sum()) # 缺失值记录数    
    print('Dtypes', each_data.dtypes) # 数据类型

[data summary for =============2015===============]
Overview: 
           会员ID         订单号       提交日期    订单金额 max_year_date
0  15278002468  3000304681 2015-01-01   499.0    2015-12-31
1  39236378972  3000305791 2015-01-01  2588.0    2015-12-31
2  38722039578  3000641787 2015-01-01   498.0    2015-12-31
3  11049640063  3000798913 2015-01-01  1572.0    2015-12-31
DESC: 
                会员ID           订单号           订单金额
count  3.057400e+04  3.057400e+04   30574.000000
mean   2.921327e+10  4.020442e+09     967.270965
std    1.384598e+10  2.630518e+08    2073.397861
min    2.670000e+02  3.000305e+09       1.500000
25%    1.961657e+10  3.885746e+09      59.700000
50%    3.754532e+10  4.117491e+09     142.000000
75%    3.923630e+10  4.234853e+09     899.000000
max    3.954613e+10  4.282025e+09  111750.000000
NA records 0
Dtypes 会员ID                      int64
订单号                       int64
提交日期             datetime64[ns]
订单金额                    float64
max_year_date    datetime64[ns]
dtype: 

**合并 2016-2018 这四年的订单数据**

In [11]:
# 汇总所有数据
data_merge = pd.concat(sheet_datas[:-1])
data_merge

,会员ID,订单号,提交日期,订单金额,max_year_date
0,15278002468,3000304681,2015-01-01,499.0,2015-12-31
1,39236378972,3000305791,2015-01-01,2588.0,2015-12-31
2,38722039578,3000641787,2015-01-01,498.0,2015-12-31
3,11049640063,3000798913,2015-01-01,1572.0,2015-12-31
4,35038752292,3000821546,2015-01-01,10.1,2015-12-31
...,...,...,...,...,...
81344,39229485704,4354225182,2018-12-31,249.0,2018-12-31
81345,39229021075,4354225188,2018-12-31,89.0,2018-12-31
81346,39288976750,4354230034,2018-12-31,48.5,2018-12-31
81347,26772630,4354230163,2018-12-31,3196.0,2018-12-31


**对订单数据进行处理，增加两列：year、date_interval**

In [13]:
# 获取各自年份数据
data_merge['date_interval'] = data_merge['max_year_date'] - data_merge['提交日期']
data_merge['year'] = data_merge['提交日期'].dt.year
# 转换日期间隔为数字
data_merge['date_interval'] = data_merge['date_interval'].apply(lambda x: x.days)
data_merge

,会员ID,订单号,提交日期,订单金额,max_year_date,date_interval,year
0,15278002468,3000304681,2015-01-01,499.0,2015-12-31,364,2015
1,39236378972,3000305791,2015-01-01,2588.0,2015-12-31,364,2015
2,38722039578,3000641787,2015-01-01,498.0,2015-12-31,364,2015
3,11049640063,3000798913,2015-01-01,1572.0,2015-12-31,364,2015
4,35038752292,3000821546,2015-01-01,10.1,2015-12-31,364,2015
...,...,...,...,...,...,...,...
81344,39229485704,4354225182,2018-12-31,249.0,2018-12-31,0,2018
81345,39229021075,4354225188,2018-12-31,89.0,2018-12-31,0,2018
81346,39288976750,4354230034,2018-12-31,48.5,2018-12-31,0,2018
81347,26772630,4354230163,2018-12-31,3196.0,2018-12-31,0,2018


**汇总每1年每个会员RFM的原始数据**

In [15]:
# 按年份和会员 ID 进行汇总
rfm_gb = data_merge.groupby(['year', '会员ID'], as_index=False).agg({
    'date_interval': 'min',  # 计算最近一次订单时间
    '提交日期': 'count', # 计算订单频率
    '订单金额': 'sum'}) # 计算订单总金额

# 重命名列
rfm_gb.columns = ['year', '会员ID', 'r', 'f', 'm']
rfm_gb.head()

,year,会员ID,r,f,m
0,2015,267,197,2,105.0
1,2015,282,251,1,29.7
2,2015,283,340,1,5398.0
3,2015,343,300,1,118.0
4,2015,525,37,3,213.0


# RFM 计算

**确定 RFM 划分区间的边界值**

In [16]:
rfm_gb.iloc[:, 2:].describe().T

,count,mean,std,min,25%,50%,75%,max
r,148591.0,165.524043,101.988472,0.0,79.0,156.0,255.0,365.0
f,148591.0,1.365002,2.626953,1.0,1.0,1.0,1.0,130.0
m,148591.0,1323.741329,3753.906883,1.5,69.0,189.0,1199.0,206251.8


**定义区间边界**

In [17]:
# 定义区间边界
r_bins = [-1, 79, 255, 365] # 注意起始边界小于最小值
f_bins = [0, 2, 5, 130] 
m_bins = [0, 69, 1199, 206252]

**将 r、f、m 这 3 列划分区间**

In [19]:
# RFM 计算过程
rfm_gb['r_score'] = pd.cut(rfm_gb['r'], r_bins, labels=[3, 2, 1])
rfm_gb['f_score'] = pd.cut(rfm_gb['f'], f_bins, labels=[1, 2, 3])
rfm_gb['m_score'] = pd.cut(rfm_gb['m'], m_bins, labels=[1, 2, 3])
rfm_gb

,year,会员ID,r,f,m,r_score,f_score,m_score
0,2015,267,197,2,105.0,2,1,2
1,2015,282,251,1,29.7,2,1,1
2,2015,283,340,1,5398.0,1,1,3
3,2015,343,300,1,118.0,1,1,2
4,2015,525,37,3,213.0,3,2,2
...,...,...,...,...,...,...,...,...
148586,2018,39538034299,272,1,49.0,1,1,1
148587,2018,39538034662,189,1,3558.0,2,1,3
148588,2018,39538035729,179,1,3699.0,2,1,3
148589,2018,39545237824,275,1,49.0,1,1,1


**将r_score、f_score、m_score三列转换为str类型**

In [20]:
# 计算 RFM 组合
rfm_gb['r_score'] = rfm_gb['r_score'].astype(np.str)
rfm_gb['f_score'] = rfm_gb['f_score'].astype(np.str)
rfm_gb['m_score'] = rfm_gb['m_score'].astype(np.str)

**将 r_score、f_score、m_score这3列的内容进行拼接**

In [21]:
rfm_gb['rfm_group'] = rfm_gb['r_score'].str.cat(rfm_gb['f_score']).str.cat(rfm_gb['m_score'])
rfm_gb

,year,会员ID,r,f,m,r_score,f_score,m_score,rfm_group
0,2015,267,197,2,105.0,2,1,2,212
1,2015,282,251,1,29.7,2,1,1,211
2,2015,283,340,1,5398.0,1,1,3,113
3,2015,343,300,1,118.0,1,1,2,112
4,2015,525,37,3,213.0,3,2,2,322
...,...,...,...,...,...,...,...,...,...
148586,2018,39538034299,272,1,49.0,1,1,1,111
148587,2018,39538034662,189,1,3558.0,2,1,3,213
148588,2018,39538035729,179,1,3699.0,2,1,3,213
148589,2018,39545237824,275,1,49.0,1,1,1,111


# RFM 结果保存

**将 rfm 的结果保存到 excel 文件**

In [23]:
rfm_group = rfm_gb['rfm_group']
rfm_group

0         212
1         211
2         113
3         112
4         322
         ... 
148586    111
148587    213
148588    213
148589    111
148590    211
Name: rfm_group, Length: 148591, dtype: object

In [24]:
rfm_group.to_excel('./data/sales_rfm_score.xlsx', index=False)

**将 rfm 的结果保存到 sales_rfm_score 表中**

In [25]:
# 写入数据到数据库
config = {'host': '127.0.0.1',  # 默认127.0.0.1
          'user': 'root',  # 用户名
          'password': '123456',  # 密码
          'port': 3306,  # 端口，默认为3306
          'database': 'rfm_db',  # 数据库名称
          'charset': 'utf8'  # 字符编码
          }

# 建表操作
con = pymysql.connect(**config)  # 建立mysql连接
cursor = con.cursor()  # 获得游标
cursor.execute("show tables")  # 查询表
table_list = [t[0] for t in cursor.fetchall()]  # 读出所有库
# 查找数据库是否存在目标表，如果没有则新建
table_name = 'sales_rfm_score'  # 要写库的表名
if not table_name in table_list:  # 如果目标表没有创建
    cursor.execute('''
    CREATE TABLE %s (
    userid               VARCHAR(20),
    r_score              int(2),
    f_score              int(2),
    m_score              int(2),
    rfm_group            VARCHAR(10),
    insert_date          VARCHAR(20)
    )ENGINE=InnoDB DEFAULT CHARSET=utf8
    ''' % table_name)  # 创建新表

In [26]:
# 梳理数据
write_db_data = rfm_gb[['会员ID','r_score','f_score','m_score','rfm_group']] # 主要数据
timestamp = time.strftime('%Y-%m-%d', time.localtime(time.time()))  # 日期

# 写库
for each_value in write_db_data.values:
    insert_sql = "INSERT INTO `%s` VALUES ('%s',%s,%s,%s,'%s','%s')" % \
                 (table_name, each_value[0], each_value[1], each_value[2], \
                  each_value[3],each_value[4],
                  timestamp)  # 写库SQL依据
    cursor.execute(insert_sql)  # 执行SQL语句，execute函数里面要用双引号
    con.commit()  # 提交命令
cursor.close()  # 关闭游标
con.close()  # 关闭数据库连接

# RFM 结果可视化

**使用 pyecharts 绘制 3D 柱形图**

In [27]:
display_data = rfm_gb.groupby(['rfm_group', 'year'], as_index=False)['会员ID'].count()
display_data.columns = ['rfm_group', 'year', 'number']
display_data['rfm_group'] = display_data['rfm_group'].astype(np.int32)
display_data.head()

,rfm_group,year,number
0,111,2015,2180
1,111,2016,1498
2,111,2017,3169
3,111,2018,2271
4,112,2015,3811


In [28]:
from pyecharts.commons.utils import JsCode
from pyecharts import options as opts

range_color = ['#313695', '#4575b4', '#74add1', '#abd9e9', '#e0f3f8', '#ffffbf',
               '#fee090', '#fdae61', '#f46d43', '#d73027', '#a50026']
range_max = int(display_data['number'].max())

# 绘制 3D 图形
bar = Bar3D().add(
    series_name='',
    data=[d.tolist() for d in display_data.values],
    xaxis3d_opts=opts.Axis3DOpts(type_='category', name='分组名称'),
    yaxis3d_opts=opts.Axis3DOpts(type_='category', name='年份'),
    zaxis3d_opts=opts.Axis3DOpts(type_='value', name='会员数量')
).set_global_opts(
    visualmap_opts=opts.VisualMapOpts(max_=range_max, range_color=range_color),
    title_opts=opts.TitleOpts(title="RFM分组结果")
)

bar.render_notebook()